## Importing data and required libraries

### Importing necessary libraries

In [15]:
import numpy as np 
import pandas as pd

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

%matplotlib inline

### Checking the directories of the datasets available

In [16]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/santander-value-prediction-challenge/train.csv
/kaggle/input/santander-value-prediction-challenge/test.csv
/kaggle/input/santander-value-prediction-challenge/sample_submission.csv


### Importing the dataset

In [17]:
dataset = pd.read_csv("../input/santander-value-prediction-challenge/train.csv")
dataset.shape

(4459, 4993)

### Printing the first five records

In [18]:
dataset.head(5)

,ID,target,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
0,000d6aaf2,38000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
1,000fbd867,600000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
2,0027d6b71,10000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
3,0028cbf45,2000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
4,002a68644,14400000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0


## Data Exploration

### Statistics of the data

In [19]:
dataset.describe()

,target,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,20aa07010,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
count,4.459000e+03,4.459000e+03,4.459000e+03,4.459000e+03,4.459000e+03,4.459000e+03,4.459000e+03,4.459000e+03,4.459000e+03,4.459000e+03,...,4.459000e+03,4.459000e+03,4.459000e+03,4.459000e+03,4459.000000,4.459000e+03,4.459000e+03,4.459000e+03,4.459000e+03,4.459000e+03
mean,5.944923e+06,1.465493e+04,1.390895e+03,2.672245e+04,4.530164e+03,2.640996e+04,3.070811e+04,1.686522e+04,4.669208e+03,2.569407e+06,...,4.676057e+05,4.446239e+05,8.056219e+05,7.812966e+05,143.529939,1.213809e+05,3.573451e+04,3.123741e+05,9.219960e+04,2.279100e+05
std,8.234312e+06,3.893298e+05,6.428302e+04,5.699652e+05,2.359124e+05,1.514730e+06,5.770590e+05,7.512756e+05,1.879449e+05,9.610183e+06,...,4.068038e+06,4.428889e+06,4.513246e+06,6.839451e+06,9584.318507,4.720709e+06,1.614622e+06,4.318501e+06,1.635993e+06,1.811139e+06
min,3.000000e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,6.000000e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.260000e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,8.000000e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.000000e+05,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,4.000000e+07,2.000000e+07,4.000000e+06,2.000000e+07,1.480000e+07,1.000000e+08,2.070800e+07,4.000000e+07,1.040000e+07,3.196120e+08,...,7.600000e+07,1.235880e+08,1.300000e+08,1.444000e+08,640000.000000,3.013120e+08,1.064200e+08,1.400000e+08,6.176800e+07,4.320000e+07


### Checking the null values in the data

In [ ]:
dataset.isnull().sum()

### Checking the data type of the columns

In [ ]:
dataset.dtypes

### Listing all the datatypes used in the dataset

In [ ]:
dataset.dtypes.value_counts()

### Removing constant columns

In [20]:
columns = dataset.std() == 0
const_columns = columns.iloc[[i for i, x in enumerate(columns) if x]]
dataset.drop(const_columns.index, axis = 1, inplace = True)

### Removing sparse columns

In [21]:
def drop_sparse(train):
    sparse_columns = []
    flist = dataset.columns[2:]
    for f in flist:
        if len(np.unique(train[f]))<2:
            sparse_columns.append(f)
    return sparse_columns

sparse_columns = drop_sparse(dataset)
dataset.drop(sparse_columns, axis = 1, inplace = True)

### Removing duplicate columns

In [55]:
def duplicate_columns(dataset):
    groups = dataset.columns.to_series().groupby(dataset.dtypes).groups
    my_dict = {}
    duplicate_features = []
    
    for d_type, columns in groups.items():
        columns_group = dataset[columns]
        list_of_column_names = dataset[columns].columns
        length = len(columns)
        
        for i in range(length):
            a = tuple(columns_group.iloc[:, i])
            if a in my_dict:
                duplicate_features.append(list_of_column_names[i])
            else:
                my_dict[a] = list_of_column_names[i]
            
    return duplicate_features

duplicate_features = duplicate_columns(dataset)
dataset.drop(sparse_columns, axis = 1, inplace = True)

## Model Training

### XGBoost

In [60]:
dataset = dataset.drop(['ID'], axis = 1)
X = dataset.iloc[:,1:]       
Y = dataset.iloc[:,0]  

In [61]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.21, random_state=42)

In [62]:
xb_model = XGBRegressor(colsample_bytree=0.055, colsample_bylevel =0.5, 
                             gamma=1.5, learning_rate=0.01, max_depth=32, 
                             objective='reg:linear',booster='gbtree',
                             min_child_weight=57, n_estimators=800, reg_alpha=0, 
                             reg_lambda = 0,eval_metric = 'rmse', subsample=0.7, 
                             silent=1, n_jobs = -1, early_stopping_rounds = 14,
                             random_state =7, nthread = -1)
xb_model.fit(x_train, y_train,
             eval_set=[(x_test, y_test)], verbose=False)

/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.5,
             colsample_bynode=1, colsample_bytree=0.055,
             early_stopping_rounds=14, eval_metric='rmse', gamma=1.5,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=32, min_child_weight=57, missing=None, n_estimators=800,
             n_jobs=-1, nthread=-1, objective='reg:linear', random_state=7,
             reg_alpha=0, reg_lambda=0, scale_pos_weight=1, seed=None, silent=1,
             subsample=0.7, verbosity=1)

In [65]:
# Predicting the values form the test_set
y_pred = xb_model.predict(x_test)

# the root-mean squared error for predictions
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmse

6472258.270427615

In [66]:
# Comparing the predictions rmse with benchmark rmse
y_mean = [y_test.mean()] * y_test.shape[0]

rmse_benchmarch = np.sqrt(mean_squared_error(y_mean, y_pred))
print(rmse_benchmarch)

4147856.2977376375


In [68]:
# Loading the test dataset for predicting the target feature
test_dataset = pd.read_csv("../input/santander-value-prediction-challenge/test.csv")
test_id_s = test_dataset['ID']
test_dataset = test_dataset[dataset.columns[1:]]
test_pred_xgb = xb_model.predict(test_dataset)
test_pred_xgb = np.clip(test_pred_xgb, 1, float('inf'))

### CatBoost

In [69]:
cb_model = CatBoostRegressor(iterations=1000,
                             learning_rate=0.05,
                             depth=10,
                             eval_metric='RMSE',
                             random_seed = 42,
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             metric_period = 50,
                             od_wait=20)

In [70]:
cb_model.fit(x_train, y_train,
             eval_set=(x_test, y_test),
             use_best_model=True,
             verbose=50)

0:	learn: 8311366.4128798	test: 7619986.4909407	best: 7619986.4909407 (0)	total: 2.08s	remaining: 34m 37s
50:	learn: 6995051.0155254	test: 6727565.4464549	best: 6727565.4464549 (50)	total: 1m 37s	remaining: 30m 20s
100:	learn: 6601671.3306085	test: 6616971.9021352	best: 6614182.5275583 (98)	total: 3m 15s	remaining: 28m 58s
150:	learn: 6359982.9780276	test: 6585429.5980834	best: 6585369.0309191 (146)	total: 4m 52s	remaining: 27m 26s
200:	learn: 6081030.4533234	test: 6576062.6011977	best: 6571619.9557888 (194)	total: 6m 29s	remaining: 25m 46s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 6564198.07
bestIteration = 223

Shrink model to first 224 iterations.


In [71]:
test_pred_cb = cb_model.predict(test_dataset)
test_pred_cb = np.clip(test_pred_cb, 0, float('inf'))

In [72]:
final_preds = (test_pred_xgb * 0.5 + test_pred_cb * 0.3)
pred_df = pd.DataFrame({'ID':test_id_s, 'target': final_preds})
pred_df.to_csv("submission.csv", float_format="%.10g", index = False)